# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](https://networkx.org/documentation/stable/tutorial.html). For this homework, you may only use the basic undirected graph methods listed [here](https://networkx.org/documentation/stable/reference/classes/graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [39]:
import networkx
homer = open('homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

Next implement a function to read in the edges from the input file.

In [40]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A list of the nodes in the graph of the form:
            ['CH', 'AG, 'ME', ...]
    """
    #To set the marker back at the starting position
    homer.seek(0)
    # List to store nodes
    nodes = []
    for line in gfile:
        line = line.strip()
        #to ignore the comments
        if line.startswith('*'):
            continue
        # Splitting Sections based on empty spaces
        sections = line.split(' ')
        for section in sections:
            # for removing 1:, &: etc
            section_parts = section.split(':')
            # The if loop will take the [1] element hence eliminating &: , 1:, etc.
            if len(section_parts)>1:
                section = section_parts[1]
                elements = section.split(';')
                #To find nodes
                for element in elements:
                    chars = element.split(',')
                    nodes.extend(chars)
    nodes = [node for node in nodes if node]# to remove empty spaces
    #To remove duplicates, I convert it ot set, since python has unique sets and then return it again as a list
    nodes = list(set(nodes)) 
    
    return nodes
    

In [41]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        The edges in the graph as a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    #To set the marker back at the starting position
    homer.seek(0)
    #intializing edges list
    edges = []
    for linez in gfile:
        linez=linez.strip()
        #To ignore the commments
        if linez.startswith('*'):
            continue
        # Splitting Sections based on empty spaces
        sectionz = linez.split(' ')
        
        for sectioni in sectionz:
            # for removing 1:, &: etc
            section_partz = sectioni.split(':')
            # The if loop will take the [1] element hence eliminating &: , 1:, etc.
            if len(section_partz) > 1:
                sectioni = section_partz[1]
                
                #spliting the lines based on ;
                nodez = sectioni.split(';')
                for nodei in nodez:
                # Split each node into individual to create a list of edges
                    elementz = nodei.split(',')
                    # Running the loop to get the edges
                    for i in range(len(elementz)):
                        for j in range( i+1, len(elementz)): #To avoid backedges
                            
                            edge = (elementz[i], elementz[j])
                            edges.append(edge)
    #The dataset has repeat edges i.e. some edges for example AC and GC are there twice but we don't need to worry about it since the undirected graph function takes care
    #To remove Repeats: I convert it ot set, since python has unique sets and then return it again as a list                     
    edges=list(set(edges))
    return edges


The following code should now correctly create the graph.

In [49]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))
print(len(G.nodes))

561


Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [43]:
def Search(graph, root):
    """Runs Search from vertex root in a graph. Neighboring nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    visited = []  # List to store visited nodes
    stack = [root]  # Stack to keep track of nodes to visit next
    while stack:
        # Get the last node from the stack
        node = stack.pop()  
        if node not in visited:
            # Mark the node as visited
            visited.append(node)  
            # Get neighbors in Reverse alphabetical order so that when we send it into stack it will use LIFO thus selecting the smallest node in the alphabetical order defined above
            neighbors = sorted(graph.neighbors(node), reverse = True)  
            # Add the reverse sorted neighbors to the stack
            stack.extend(neighbors) 
             
    return visited
    

We will check the correctness of your code by verifying that it correctly computes the connected component of node Ulysses (node `OD`).

In [50]:
ulysses = Search(G, 'OD')
print(ulysses)

['OD', '0X', '1M', '1N', '1O', '1P', '1Q', '1R', '1S', '23', '2X', 'DI', '03', '04', '05', 'AG', '0D', '0E', '1H', '26', '32', '33', '34', '35', 'AC', '0M', '0L', '2I', 'AH', '2J', 'HT', '02', '01', 'AE', '0F', '07', '08', '0G', '0H', '0I', '0J', '0K', '3V', 'AJ', '0W', '1J', '1U', '3N', '3O', '3P', '3Q', '3R', '47', '4P', '4V', '4Y', '5T', '6M', '6N', '8W', '8T', '8U', '8V', '8X', 'NE', '5A', '57', 'PO', '1K', '0P', '0N', '0O', '0Q', '0R', '0S', 'AA', 'AF', 'AI', '3T', '4C', 'AO', '31', 'PD', '2W', '4H', '4I', '4J', '4K', '4L', 'PS', '36', '4G', '0C', '1I', 'AL', '06', '3W', '3X', '3Y', '3Z', '40', '93', '44', '45', '46', '92', '94', '22', '41', '42', '43', '95', '6Y', '96', '98', '9A', '9B', '9C', 'AX', '4N', '6R', 'MT', '6I', '6J', '6O', '6Q', 'TU', '24', '2N', '2O', '2P', '2Q', '2R', '2S', '2T', '2U', '2V', '48', '49', 'PR', '0T', '0U', '4A', '4B', 'ID', '11', '59', '7S', '7R', '7T', '9W', '9V', '9X', 'AR', '2C', '2B', '2A', 'BL', '29', '83', 'ZE', '1F', '1G', 'OG', '87', 'DT', 'SE

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [45]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    #a list to store the connected components
    components = []

    #a set to keep track of visited nodes
    visited = set()

    #all nodes in the graph
    #sorting them to pick the smallest unvisited node first
    all_nodes = sorted(graph.nodes())

    for node in all_nodes:
        if node not in visited:
            #performing DFS using the search function
            #search function gets all the connected nodes
            component = Search(graph, node)
            
            #adding the connected components to components list
            components.append(component)
            # Marking node as visited
            visited.update(component)
    
    return components


We will check correctness of your code by verifying that your output is identical to our solution.

In [46]:
character_interactions = connected_components(G)


As a preliminary check, you should find that the following statements are all true.

In [47]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:{}".format(len(component_sizes) == 12))
print("The giant component has size 542:{}".format(max(component_sizes) == 542))
print("There are 5 isolated characters:{}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad:True
The giant component has size 542:True
There are 5 isolated characters:True
